In [ ]:
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit

In [ ]:
# read csv
path = './data/'

def get_csv(path):
    # load CSV & drop zeros
    df = pd.read_csv(path+'driving_log.csv')
    df = df.drop(df[df['steering'] == 0.].index)
    df = df.drop(df[df['speed'] < 1.].index)
    return df

df = get_csv(path = path)


In [ ]:
# split
split_index = int((1-(1/11))*(len(df)))
df_train = df[:split_index]
df_val  = df[split_index:]

In [ ]:
# generate batch

from sklearn.utils import shuffle

image_shape = cv2.imread( path+df['center'][df['center'].index[0]]).shape
batch_size = 7 # 3675 = 3*5*5*7*7

def get_batch(df,batch_size): 
    n_split = int(len(df)//batch_size)-1
    batch_idx = (df[:batch_size*n_split].index.values)   
    batch_idx = shuffle(batch_idx)
    batch_idx = batch_idx.reshape(n_split,batch_size,)
    while 1:
        for idx in batch_idx:
            batch_y_c      =      df['steering'][idx].values
            batch_y_r      =      df['steering'][idx].values - 0.02
            batch_y_l      =      df['steering'][idx].values + 0.02
            batch_y = np.concatenate([batch_y_c,batch_y_r,batch_y_l])
            batch_y = np.concatenate([batch_y,-batch_y])

            batch_x_path_c = path+df['center'][idx]
            batch_x_path_r = path+df['right'][idx]
            batch_x_path_l = path+df['left'][idx]
            batch_x_path = np.concatenate([batch_x_path_c,batch_x_path_r,batch_x_path_l])

            batch_x      = np.empty(list([batch_size*3]) + list(image_shape))
            ii = 0
            for img_path in batch_x_path:
                image = cv2.imread(img_path)
                #image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS) ## COLOR_RGB2HSV COLOR_RGB2HLS
                batch_x[ii] = image
                ii +=1
                

            batch_x = np.concatenate([batch_x,batch_x[:,:,-1::-1]])


            assert(not(np.isnan((np.sum(batch_x)))))
            assert(not(np.isnan((np.sum(batch_y)))))
            yield shuffle(batch_x,batch_y) 

train_generator = get_batch(df_train, batch_size=batch_size)
validation_generator = get_batch(df_val, batch_size=batch_size)

batch_x,batch_y = next(train_generator)

In [ ]:
def visualise_dataset(image,label,steps):
    plt_num = 1
    print(image.shape)
    step_size = int(image.shape[0]/steps)
    for image_idx in range(0,image.shape[0],step_size):
        channels = image.shape[3]
        plt.figure(plt_num, figsize=(32,32))
        for channel in range(channels):
            plt.subplot(4,8, channel+1) # sets the number of feature maps to show on each row and column
            plt.title('channel ' + str(channel)) # displays the feature map number
            plt.imshow(image[image_idx,:,:, channel], interpolation="nearest", cmap="gray")
        plt.show()
        print('{1}, idx: {0}'.format(image_idx,label[image_idx],))
        
#%time visualise_dataset(batch_x,batch_y,steps=6)

In [ ]:
from keras.models import Sequential,load_model
from keras.layers.core import Dense, Activation, Dropout, Reshape, Flatten, Lambda
from keras.layers.convolutional import Convolution2D, Cropping2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers import Merge

import tensorflow as tf
import matplotlib

# Hyperparameter Compile
loss= 'mse' # 'hinge'
optimizer= 'Nadam' #'Nadam' #'rmsprop'

# Hyperparameter Fit
nb_epoch= 10 * 5

try:
    model = load_model('model.h5')
    print("load_model done")
except:
    print("load_model failed")
    
def model_1():
    model = Sequential()
    model.add(Cropping2D(cropping=((50,25), (0,0)), input_shape=image_shape))
    model.add(Lambda(lambda x: (x[:,:,:,0:1]+x[:,:,:,1:2]+x[:,:,:,2:3])/3)) ## drive on red
    model.add(Lambda(lambda x: x/127.5 - 1.))
    return model

def model_1hsv():
    model = Sequential()
    model.add(Cropping2D(cropping=((50,25), (0,0)), input_shape=image_shape))
    model.add(Lambda(lambda x: matplotlib.colors.rgb_to_hsv(x)))
    model.add(Lambda(lambda x: (x[:,:,:,1:3]))) 
    return model

def model_1mirror():
    model1 = model_1()
    model2 = model1
    model2.add(Lambda(lambda x: x[:,:,-1::-1]))
    model2.add(Lambda(lambda x: matplotlib.colors.rgb_to_hsv(x)))

    model = Sequential()
    model.add(Merge([model2, model1]))
    return model

def model_3():
    model = model_1()
    model.add(Convolution2D(6,6,1,activation='relu'))
    model.add(MaxPooling2D())
    #model.add(BatchNormalization())
    model.add(Convolution2D(6,6,1,activation='relu'))
    model.add(MaxPooling2D())
    #model.add(BatchNormalization())
    model.add(Convolution2D(6,6,1,activation='relu'))    
    model.add(MaxPooling2D())
    model.add(Flatten())
    model.add(Dense(300))
    #model.add(Dropout(0.5))
    model.add(Dropout(0.90))
    model.add(Dense(120))
    model.add(Dense(84))
    #model.add(Dropout(0.5))
    model.add(Dense(1))
    model.add(Activation('linear'))
    return model

model = model_3()


%time model.compile(loss=loss, optimizer=optimizer)
%time history_object = model.fit_generator(generator         = train_generator, \
                    samples_per_epoch = batch_size*6, \
                    validation_data   = validation_generator, \
                    nb_val_samples    = 1, \
                    nb_epoch          = nb_epoch)

model.save("model.h5")
print('model saved')
#model = load_model('model.h5')

### print the keys contained in the history object
print(history_object.history.keys())

### plot the training and validation loss for each epoch
plt.plot(history_object.history['loss'])
plt.plot(history_object.history['val_loss'])
plt.title('model mean squared error loss')
plt.ylabel('mean squared error loss')
plt.xlabel('epoch')
plt.legend(['training set', 'validation set'], loc='upper right')
plt.yscale('log')
plt.show()


#model.save("model.h5")

model = load_model('model.h5')



In [ ]:
'''
def get_StratifiedShuffleSplit(batches_idx,batch_size):
    n_bins  = 5
    batch_size = 5*7*7
    y_classes = df['steering'].copy()*n_bins//1
    batches_idx = df.index
    data_size = len(batches_idx)
    n_split = int(data_size//batch_size)-1
    print(n_split)
    batch_range = df.index
    sss = StratifiedShuffleSplit(n_splits = n_split,test_size = 15)
    batch_idx, val_idx = next(sss.split((batch_range),(y_classes.values)))
    
    batch_idx = batch_idx[:batch_size*n_split].reshape((n_split,batch_size, ))
    batch_idx = batch_idx[:batch_size*n_split].reshape((n_split,batch_size, ))
    
    return batch_idx, val_idx

    batch_idx, val_idx = (get_batch_idx(batches_idx,batch_size = 256))
    batch_idx.shape
    
    #from keras.layers.recurrent import LSTM, GRU, SimpleRNN
#from keras.layers.wrappers import TimeDistributed
#from keras.regularizers import l2, activity_l2


'''